In [1]:
import pandas as pd
import os
import re
import warnings

In [2]:
file_path = 'C:/Users/htet.ma\\Downloads\\MP Inspected'
file_names = os.listdir(file_path)

# all_sheets = {}
all_sheets_xls = []

for name in file_names:
    first_sheet = pd.read_excel(file_path + "\\" + str(name), sheet_name=0, header=0, skiprows=[1])
    # csv_filename = os.path.splitext(name)[0] + '.csv'  # Create the corresponding CSV filename
    # first_sheet.to_csv(csv_filename, index=False)  # Save the sheet as a CSV file
    all_sheets_xls.append(first_sheet)
#     all_sheets[name] = first_sheet

combined_xls = pd.concat(all_sheets_xls, ignore_index=True)

print(f'finished')

finished


In [40]:
# Dictionary

# Repair
repair_all = {'Everything': ['509A.14', '513E.01', '509A.26', '1509A.06', '1509A.04', '1509A.05', '513A.01']}
repair_cell = {'170': ['504A.19', '508I.52', '508I.53', '508I.44', '509A.18', 'RAG10003', 'RDI10004'], 
              '190': ['504A.19', '508I.52', '508I.53', '508I.44', '509A.18', 'RAG10003', 'RDI10004'], 
              '1100': ['504A.19', '508I.52', '508I.53', '508I.44', '509A.18', 'RAG10010', 'RDI10008'], 
              '1500': ['504A.19', '508I.52', '508I.53', '508I.44', '509A.18', 'RAG10010', 'RDI10008'], 
              'Everything': ['504A.19', '508I.52', '508I.53', '508I.44', '509A.18', '3001E.22', '404P.04']}

# Refurbishment
refurb = {'TJF': ['509A.21', '508I.53', '513E.01', '509A.26', '508I.52', '502H.14', '513A.01', '508I.44', '509A.18', '3000R.29', '509A.17'], 
          'JF': ['509A.21', '508I.53', '513E.01', '509A.26', '508I.52', '502H.14', '513A.01', '508I.44', '509A.18', '3000R.29', '509A.17'], 
          'PJF': ['509A.21', '508I.53', '513E.01', '509A.26', '508I.52', '502H.14', '513A.01', '508I.44', '509A.18', '3000R.29', '509A.17'], 
          'Everything': ['509A.21', '508I.53', '513E.01', '509A.26', '508I.52', '502H.14', '513A.01', '508I.44', '509A.18', 'N/AForceps', 'N/APull']}

print(f'finished')

finished


In [4]:
def get_chapters(model_generation, dictionary_group, group_repair):
    chapters = []
    
    if model_generation in dictionary_group.keys():
        for value in dictionary_group[model_generation]:
            if value in group_repair['Work Instruction Document Number '].values:
                chapters.append(0)
            else:
                chapters.append(value)
    else:
        for value in dictionary_group['Everything']:
            if value in group_repair['Work Instruction Document Number '].values:
                chapters.append(0)
            else:
                chapters.append(value)
            
    return chapters

In [5]:
def get_model(model_name):
    for index, char in enumerate(model_name):
        if char == '-':
            split = model_name.split('-')
            split_num = re.findall(r'\d+', split[1])
            if split_num:
                return split_num[0]
            else:
                return split[1]
    return model_name

In [6]:
#Cell 8 
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    filtering_repair = combined_xls[(combined_xls['OperationName'] == 'Repair') & (~combined_xls['LaborCode'].isin(['209A', '202A', '23A', '23N', '92A', '92H', '93A', '13A', '137C', '137D', '137E', '137F', '137I']))]

    groupping_repair = filtering_repair.groupby(['ModelFamily', 'ModelNo', 'LaborCode'])

    repair_checks = pd.DataFrame(columns=['Family', 'Model', 'LaborCode', '1', '2', '3', '4', '5', '6', '7'])

    for group_key, group_repair in groupping_repair:
    #     print("Group Key:", group_key)
    #     print(group_repair)
    #     print()

        model_number = get_model(group_key[1])

        missing_chapters = get_chapters(model_number, repair_all, group_repair)

        together = [group_key[0]] + [group_key[1]] + [group_key[2]] + missing_chapters
        repair_checks = repair_checks.append(pd.Series(together, index=repair_checks.columns), ignore_index=True)

    print(f'finished')

finished


In [7]:
#cell 6
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    filtering_repair = combined_xls[(combined_xls['OperationName'] == 'Repair') & (combined_xls['LaborCode'].isin(['05A','116A','14A','16B','16G','20A','213A','217A','38A','80A','81A','82A','83A','83B','83C','84A','85A','86A','88A']))]

    groupping_repair = filtering_repair.groupby(['ModelFamily', 'ModelNo', 'LaborCode'])

    repair_checks_cell6 = pd.DataFrame(columns=['Family', 'Model', 'LaborCode', '1', '2', '3', '4', '5', '6', '7'])

    for group_key, group_repair in groupping_repair:

        model_number = get_model(group_key[1])

        missing_chapters = get_chapters(model_number, repair_cell, group_repair)

        together = [group_key[0]] + [group_key[1]] + [group_key[2]] + missing_chapters
        repair_checks_cell6 = repair_checks_cell6.append(pd.Series(together, index=repair_checks.columns), ignore_index=True)

    print(f'finished')

finished


In [8]:
repair_final = pd.merge(repair_checks, repair_checks_cell6, on=['Family', 'Model', 'LaborCode'], how='outer', indicator=True)

output_file_path = 'C:/Users/htet.ma\\Downloads\\repair_final.xlsx'
repair_final.to_excel(output_file_path, index=False)

print(f'finished')

finished


In [29]:
def get_refurb_model(model_name):
    if '-' in model_name:
        split = model_name.split('-')
        return split[0]
    else:
        return model_name

In [43]:
#Refurbishment Operations
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    
    filtered_refurb = combined_xls[(combined_xls['OperationName'] == 'Assembly') & (combined_xls['LaborCode'].isin(['28A', '28D', '28F']))]

    groupping_refurb = filtered_refurb.groupby(['ModelFamily', 'ModelNo', 'LaborCode'])

    refurb_checks = pd.DataFrame(columns=['Family', 'Model', 'LaborCode', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11'])

    for group_key, group_repair in groupping_refurb:
    #     print("Group Key:", group_key)
    #     print(group_repair)
    #     print()

        model_number = get_refurb_model(group_key[1])

        missing_chapters = get_chapters(model_number, refurb, group_repair)

        together = [group_key[0]] + [group_key[1]] + [group_key[2]] + missing_chapters
        refurb_checks = refurb_checks.append(pd.Series(together, index=refurb_checks.columns), ignore_index=True)

    print(f'finished')

finished


In [44]:
output_file_path = 'C:/Users/htet.ma\\Downloads\\refurb_final.xlsx'
refurb_checks.to_excel(output_file_path, index=False)

print(f'finished')

finished
